# Exploring Python SDK v2 in Azure Machine Learning

In this example we will explore the Python SDK (v1) and some functionalities for Machine Learning.
We will create:
- Install the required Python packages
- MLFlow tracking
- Get data from Azure storage
- Create a simple Py function
- Run the prediction models


In [2]:
# Instsalling required Py Packages
#!pip install azure-ai-ml
%pip install -r RequirementsPySDK.txt

     |████████████████████████████████| 12.2 MB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 77.6 MB/s eta 0:00:01
ERROR: pyldavis 3.3.1 requires sklearn, which is not installed.
ERROR: pandas-ml 0.6.1 requires enum34, which is not installed.
ERROR: fbprophet 0.7.1 requires cmdstanpy==0.9.5, which is not installed.
ERROR: tensorboard 2.2.2 has requirement google-auth<2,>=1.6.3, but you'll have google-auth 2.13.0 which is incompatible.
ERROR: responsibleai 0.22.0 has requirement ipykernel<=6.6.0, but you'll have ipykernel 6.8.0 which is incompatible.
ERROR: responsibleai 0.22.0 has requirement markupsafe<2.1.0, but you'll have markupsafe 2.1.1 which is incompatible.
ERROR: responsibleai 0.22.0 has requirement numba<0.54.0, but you'll have numba 0.55.2 which is incompatible.
ERROR: raiwidgets 0.22.0 has requirement itsdangerous==2.0.1, but you'll have itsdangerous 2.1.2 which is incompatible.
ERROR: rai-core-flask 0.4.0 has requirement gevent==21.12.0, but you'll h

In [3]:
#import mlflow and set tracking and import Workspace
import mlflow
from azureml.core import Workspace

ws = Workspace.from_config()

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment("lightgbm-iris-toy-demo")
     

2022/12/07 05:12:34 INFO mlflow.tracking.fluent: Experiment with name 'lightgbm-iris-toy-demo' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1670389955391, experiment_id='cb09a125-b165-4a68-9525-fb606f6a5837', last_update_time=None, lifecycle_stage='active', name='lightgbm-iris-toy-demo', tags={}>

In [4]:
# get the data from local Azure Blob store
# this is a general location for the data. 
data_uri = "https://azuremlexamples.blob.core.windows.net/datasets/iris.csv"

In [5]:
#convert CSV to pandas and check the first couple of rows
import pandas as pd

df = pd.read_csv(data_uri)
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Now, let's create some Python functions for machine learning. These will be:
- preprocess_data
- train_model
- evaluate_model

I am importing also sklearn metrics functions - log_loss and accuracy_score, as well as the sklearn train_test split functions :-)


In [6]:
# imports
import time

import lightgbm as lgb

from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# define functions
def preprocess_data(df):
    X = df.drop(["species"], axis=1)
    y = df["species"]

    enc = LabelEncoder()
    y = enc.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    return X_train, X_test, y_train, y_test, enc


def train_model(params, num_boost_round, X_train, X_test, y_train, y_test):
    t1 = time.time()
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test)
    model = lgb.train(
        params,
        train_data,
        num_boost_round=num_boost_round,
        valid_sets=[test_data],
        valid_names=["test"],
    )
    t2 = time.time()

    return model, t2 - t1


def evaluate_model(model, X_test, y_test):
    y_proba = model.predict(X_test)
    y_pred = y_proba.argmax(axis=1)
    loss = log_loss(y_test, y_proba)
    acc = accuracy_score(y_test, y_pred)

    return loss, acc

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


We will define the parameters for Lightgbm and create a wrapper for MLFlow to start logging and evaluating the models.

In [7]:

# preprocess data
X_train, X_test, y_train, y_test, enc = preprocess_data(df)

# set training parameters
params = {
    "objective": "multiclass",
    "num_class": 3,
    "learning_rate": 0.1,
    "metric": "multi_logloss",
    "colsample_bytree": 1.0,
    "subsample": 1.0,
    "seed": 42,
}

num_boost_round = 32

# start run
run = mlflow.start_run()

# enable automatic logging
mlflow.lightgbm.autolog()

# train model
model, train_time = train_model(
    params, num_boost_round, X_train, X_test, y_train, y_test
)
mlflow.log_metric("training_time", train_time)

# evaluate model
loss, acc = evaluate_model(model, X_test, y_test)
mlflow.log_metrics({"loss": loss, "accuracy": acc})
     

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007484 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 91
[LightGBM] [Info] Number of data points in the train set: 120, number of used features: 4
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.073920
[LightGBM] [Info] Start training from score -1.123930
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	test's multi_logloss: 0.930558
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2

2022/12/07 05:46:20 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmprxlc7tbq/model, flavor: lightgbm), fall back to return ['lightgbm==3.2.1']. Set logging level to DEBUG to see the full traceback.


In [10]:
#save the logs and capture the results
mlflow.log_artifact("PySDK.ipynb")

In [11]:
# end run and mark it as complete
mlflow.end_run()